In [1]:
import os
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\kidney_disease\\research'

In [2]:
os.chdir('../')
%pwd

'd:\\Jeevan\\VSCode\\Text_summarization\\kidney_disease'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data:Path
    params_epochs:int
    params_batch_size: int
    params_image_size: list
    params_is_augmentation: bool

In [4]:
from kidney_disease.utils.common import read_yaml,create_directories
from kidney_disease.constant import *

In [5]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_training_config(self)->ModelTrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image")

        create_directories([
            Path(training.root_dir)
        ])

        return ModelTrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=self.params.EPOCHS,
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE,
            params_is_augmentation=self.params.AUGMENTATION,
        )


In [6]:
import tensorflow as tf

In [7]:
class ModelTraining:
    def __init__(self,config: ModelTrainingConfig):
        self.config=config
    
    def get_base_model(self):
        self.model=tf.keras.models.load_model(self.config.updated_base_model_path)
    
    def train_valid_generator(self):
        try:
            import PIL
            print("Pillow version:", PIL.__version__)
        except ImportError:
            print("Pillow is not installed.")
            
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.2
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerator_kwargs
            )
            self.train_generator=train_datagenerator.flow_from_directory(
                directory=self.config.training_data,
                subset='training',
                **dataflow_kwargs
            )
    @staticmethod
    def save_model(model:tf.keras.Model,path:Path):
        model.save(path)
        
    def train_model(self):
        self.steps_per_epoch=self.train_generator.samples//self.config.params_batch_size
        self.validation_steps=self.valid_generator.samples//self.config.params_batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps
        )

        self.save_model(model=self.model,path=self.config.trained_model_path)

In [8]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = ModelTraining(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train_model()
    
except Exception as e:
    raise e

[2025-04-11 00:30:27,561: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-11 00:30:27,564: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-11 00:30:27,566: INFO: common: created directory at: artifacts]
[2025-04-11 00:30:27,569: INFO: common: created directory at: artifacts\training]
Pillow version: 11.1.0
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 [==============================] - 50s 2s/step - loss: 9.5676 - accuracy: 0.5899 - val_loss: 1.8539 - val_accuracy: 0.6500
